In [1]:
import pandas as pd
import numpy as np

import utils.print as print_f

from utils.coco_eval import get_eval_params_dict
from utils.engine import xami_train_one_epoch, xami_evaluate, get_iou_types
from utils.plot import plot_losses, plot_train_val_ap_ars, get_ap_ar_for_train_val
from utils.save import get_data_from_metric_logger
from utils.coco_utils import get_cocos

from models.setup import ModelSetup
from models.build import create_model_from_setup
from models.train import TrainingInfo
from utils.save import check_best, end_train
from data.load import get_datasets, get_dataloaders

from IPython.display import clear_output
from utils.eval import get_ar_ap
from utils.train import get_optimiser, get_lr_scheduler, print_params_setup
from utils.init import reproducibility, clean_memory_get_device
from data.constants import DEFAULT_REFLACX_LABEL_COLS, XAMI_MIMIC_PATH
from  datetime import datetime
import torch.optim as optim
from models.dynamic_loss import DynamicWeightedLoss
## Suppress the assignement warning from pandas.r
pd.options.mode.chained_assignment = None  # default='warn'

## Supress user warning
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

%matplotlib inline

In [2]:
device = clean_memory_get_device()
reproducibility()


This notebook will running on device: [CUDA]


In [3]:
use_iobb = True
io_type_str = "IoBB" if use_iobb else "IoU"
labels_cols = DEFAULT_REFLACX_LABEL_COLS
iou_thrs = np.array([0.5])

In [4]:
model_setup =ModelSetup(
        name="CXR",
        use_clinical=False,
        use_custom_model=True,
        use_early_stop_model=True,
        best_ar_val_model_path=None,
        best_ap_val_model_path=None,
        final_model_path=None,
        backbone="resnet50",
        optimiser="sgd",
        lr=1e-3,
        # lr=1e-4,
        # weight_decay=0.001,
        weight_decay=0,
        pretrained=True,
        record_training_performance=True,
        dataset_mode="unified",
        image_size=512,
        backbone_out_channels=16,
        batch_size=4,
        warmup_epochs=0,
        lr_scheduler="ReduceLROnPlateau",
        # lr_scheduler=None,
        reduceLROnPlateau_factor=0.1,
        reduceLROnPlateau_patience=500,
        reduceLROnPlateau_full_stop=False,
        multiStepLR_milestones=[30, 50, 70, 90],
        multiStepLR_gamma=0.1,
        representation_size=64, # 32
        mask_hidden_layers=256,
        using_fpn=True,
        use_mask=False,
        clinical_expand_dropout_rate=0,
        clinical_conv_dropout_rate=0,
        clinical_input_channels=32,
        clinical_num_len=9,
        clinical_conv_channels=32,
        fuse_conv_channels=32,
        fuse_dropout_rate=0,
        box_head_dropout_rate=0,
        fuse_depth=4,
        fusion_strategy="add",
        fusion_residule=False,
    )

In [5]:
print_f.print_title("Preparing for the training.")

train_info = TrainingInfo(model_setup)

################ Datasets ################
dataset_params_dict = {
    "XAMI_MIMIC_PATH": XAMI_MIMIC_PATH,
    "with_clinical": model_setup.use_clinical,
    "dataset_mode": model_setup.dataset_mode,
    "bbox_to_mask": model_setup.use_mask,
    "labels_cols": labels_cols,
}

detect_eval_dataset, train_dataset, val_dataset, test_dataset = get_datasets(
    dataset_params_dict=dataset_params_dict,
)

train_dataloader, val_dataloader, test_dataloader = get_dataloaders(
    train_dataset, val_dataset, test_dataset, batch_size=model_setup.batch_size,
)

train_coco, val_coco, test_coco = get_cocos(
    train_dataloader, val_dataloader, test_dataloader
)

eval_params_dict = get_eval_params_dict(
    detect_eval_dataset, iou_thrs=iou_thrs, use_iobb=use_iobb,
)

model = create_model_from_setup(
    labels_cols,
    model_setup,
    rpn_nms_thresh=0.3,
    box_detections_per_img=10,
    box_nms_thresh=0.2,
    rpn_score_thresh=0.0,
    box_score_thresh=0.05,
)


====================Preparing for the training.====================
creating index...
index created!
creating index...
index created!
creating index...
index created!
creating index...
index created!
Load custom model
Using ResNet as backbone
Using pretrained backbone. resnet50
Not using pretrained MaksRCNN model.


In [6]:

# dynamic_loss_weight = None
dynamic_loss_weight = DynamicWeightedLoss()
dynamic_loss_weight.to(device)
print_params_setup(model)

model.to(device)

params = [p for p in model.parameters() if p.requires_grad]
if dynamic_loss_weight:
    params += [p for p in dynamic_loss_weight.parameters() if p.requires_grad]

iou_types = get_iou_types(model, model_setup)
optimizer = get_optimiser(params, model_setup)
lr_scheduler = get_lr_scheduler(optimizer, model_setup)

current_time = datetime.now()

print_f.print_title(
    f"Start training. Preparing Tooke [{ (current_time - train_info.start_t).seconds}] sec"
)

train_info.start_t = datetime.now()


[model]: 28,202,221
[model.backbone]: 26,799,296
[model.rpn]: 593,935
[model.roi_heads]: 808,990
[model.roi_heads.box_head]: 807,040
[model.roi_heads.box_head.fc6]: 802,880
[model.roi_heads.box_head.fc7]: 4,160
[model.roi_heads.box_predictor]: 1,950
Using SGD as optimizer with lr=0.001
====================Start training. Preparing Tooke [163] sec====================


In [8]:
from utils.save import save_checkpoint

save_checkpoint(
    train_info = train_info,
    model=model,
    val_ar=0,
    val_ap=0,
    test_ar=0,
    test_ap=0,
    optimizer=optimizer,
    dynamic_weight= dynamic_loss_weight,
) 

In [9]:
train_info.final_model_path

'val_ar_0_0000_ap_0_0000_test_ar_0_0000_ap_0_0000_epoch0_WithoutClincal_05-18-2022 00-03-22_CXR'

In [10]:
from enum import Enum

class TrainedModels(Enum):
    load = train_info.final_model_path
    

In [11]:
from models.load import get_trained_model

model, train_info, optimizer, dynamic_loss_weight = get_trained_model(
        TrainedModels.load,
        labels_cols,
        device,
        rpn_nms_thresh=0.3,
        box_detections_per_img=10,
        box_nms_thresh=0.2,
        rpn_score_thresh=0.0,
        box_score_thresh=0.05,
    )

Load custom model
Using ResNet as backbone
Using pretrained backbone. resnet50
Not using pretrained MaksRCNN model.
Using SGD as optimizer with lr=0.001
